**What it is:**  
- This script generates benchmark associations for real estate (REIT) portfolios by mapping portfolio names to appropriate benchmarks defined in the BenchmarkGeneralInformation (BGI) table.

**What it does:**  
- Retrieves active REIT portfolios from `PortfolioGeneralInformation`.  
- Retrieves benchmark metadata from `BenchmarkGeneralInformation`.  
- Uses keyword matching between portfolio `NAME` and benchmark `NAME` to determine relevant associations.  
- Creates a ranked list of associations and prepares it for insertion into the `PortfolioBenchmarkAssociation` table in Snowflake.

**Why it’s important:**  
- Ensures that each REIT portfolio is linked to one or more benchmarks, enabling comparative performance analysis.  
- Supports automation and scalability by reducing reliance on hard-coded rules.  
- Maintains data integrity across analytics and reporting pipelines by programmatically populating key relationships.


# **Portfolio Benchmark Association Table**

## Establishing Connection

You must download the env.txt file which is located in my GDrive folder ("Shiv-Code") and upload using the "Choose Files" widget after running the below cell.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving env.txt to env.txt


In [2]:
import os

if os.path.exists("env.txt"):
    os.rename("env.txt", "env")
    print("Renamed env.txt to env")
else:
    print("File not found. Make sure you uploaded env.txt.")

Renamed env.txt to env


In [3]:
!pip install snowflake-connector-python python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 4.5 MB/s eta 0:00:00


In [4]:
from dotenv import load_dotenv
load_dotenv("env")  # This loads the Snowflake credentials into the environment

True

In [5]:
import snowflake.connector
import os

# Pull in credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

# Run a simple test query
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE();")
for row in cur:
    print(row)

cur.close()
conn.close()

('SHIVNEETN', 'AST_REALESTATE_DB_RW', 'AST_REALESTATE_DB')


## Define Portfolio Code to Benchmark Code Mapping Logic

In [7]:
"""
Auto-generates benchmark associations for REIT portfolios by matching
keywords in portfolio names with benchmark names from BGI.
"""

import pandas as pd
from snowflake.connector import connect
import os

# Connect to Snowflake
conn = connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database="AST_REALESTATE_DB",
    schema="DBO"
)

# Step 1: Pull active REIT portfolios from PGI
"""
Fetches active REIT portfolios from PortfolioGeneralInformation.
The NAME column is assumed to embed strategy details for matching.
"""
pgi_df = pd.read_sql("""
    SELECT PORTFOLIOCODE, NAME
    FROM PortfolioGeneralInformation
    WHERE PORTFOLIOCATEGORY = 'REIT'
      AND TERMINATIONDATE IS NULL
""", conn)

# Step 2: Pull REIT benchmark names from BGI
"""
Fetches REIT-related benchmarks from BenchmarkGeneralInformation,
including the BENCHMARKCODE and full NAME for keyword comparisons.
"""
bgi_df = pd.read_sql("""
    SELECT BENCHMARKCODE, NAME AS BENCHMARKNAME
    FROM BenchmarkGeneralInformation
    WHERE BENCHMARKCODE LIKE 'REITBENCH%'
""", conn)

# Step 3: Attempt to match portfolio name keywords to benchmark names
"""
Loops through each portfolio name and attempts to find benchmark name matches.

- First tries to match the full portfolio name against benchmark names.
- If no matches are found, falls back to word-level partial matching.
- Produces an association row for each match with a calculated rank.
"""
rows = []
for _, p_row in pgi_df.iterrows():
    portfolio_code = p_row["PORTFOLIOCODE"]
    portfolio_name = p_row["NAME"]

    # Attempt full-name match
    matched_benchmarks = bgi_df[
        bgi_df["BENCHMARKNAME"].str.contains(portfolio_name, case=False, na=False)
    ]

    # Fallback: word-level match from portfolio name
    if matched_benchmarks.empty:
        words = portfolio_name.split()
        matched_benchmarks = bgi_df[
            bgi_df["BENCHMARKNAME"].apply(
                lambda x: any(word.lower() in x.lower() for word in words)
            )
        ]

    if matched_benchmarks.empty:
        print(f"No benchmark match found for portfolio name '{portfolio_name}' ({portfolio_code})")
        continue

    for rank, (_, b_row) in enumerate(matched_benchmarks.iterrows(), start=1):
        rows.append({
            "PORTFOLIOCODE": portfolio_code,
            "BENCHMARKCODE": b_row["BENCHMARKCODE"],
            "RANK": rank,
            "RECIPIENTCODE": None
        })

# Step 4: Final association DataFrame
"""
Builds the final association DataFrame (assoc_df)
from all matches, ready for downstream Snowflake upload.
"""
assoc_df = pd.DataFrame(rows)
print(f"Prepared {len(assoc_df)} benchmark associations based on portfolio NAME matching.")
print(assoc_df)


/tmp/ipython-input-2764563708.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pgi_df = pd.read_sql("""
/tmp/ipython-input-2764563708.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bgi_df = pd.read_sql("""


No benchmark match found for portfolio name 'Industrial, Retail & Infrastructure' (REITPORTFOLIO005)
Prepared 97 benchmark associations based on portfolio NAME matching.
      PORTFOLIOCODE BENCHMARKCODE  RANK RECIPIENTCODE
0   REITPORTFOLIO02   REITBENCH01     1          None
1   REITPORTFOLIO02   REITBENCH02     2          None
2   REITPORTFOLIO18   REITBENCH01     1          None
3   REITPORTFOLIO18   REITBENCH02     2          None
4   REITPORTFOLIO13   REITBENCH01     1          None
..              ...           ...   ...           ...
92      PORTFOLIO03   REITBENCH02     2          None
93      PORTFOLIO02   REITBENCH01     1          None
94      PORTFOLIO02   REITBENCH02     2          None
95      PORTFOLIO01   REITBENCH01     1          None
96      PORTFOLIO01   REITBENCH02     2          None

[97 rows x 4 columns]


## Validation

In [8]:
import logging
import pandas as pd


def validate_portfolio_benchmark_association(df: pd.DataFrame) -> pd.DataFrame:
    """
    Validate the PortfolioBenchmarkAssociation data before uploading to Snowflake.

    Checks include:
    - Required column presence
    - Nulls in key fields (PORTFOLIOCODE, BENCHMARKCODE, RANK)
    - Duplicate associations
    - RANK values being valid integers >= 1

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing portfolio-benchmark associations.

    Returns
    -------
    pd.DataFrame
        One-row summary DataFrame with validation flags and issue counts.
    """
    required_cols = ["PORTFOLIOCODE", "BENCHMARKCODE", "RANK", "RECIPIENTCODE"]
    missing_cols = [col for col in required_cols if col not in df.columns]

    if missing_cols:
        logging.error(f"[portfolio_benchmark_association] Missing required columns: {missing_cols}")
        return pd.DataFrame()

    # Validation summary
    summary = {
        "missing_portfoliocodes": df["PORTFOLIOCODE"].isnull().sum(),
        "missing_benchmarkcodes": df["BENCHMARKCODE"].isnull().sum(),
        "missing_ranks": df["RANK"].isnull().sum(),
        "invalid_rank_values": df["RANK"].apply(lambda r: not isinstance(r, (int, float)) or r < 1).sum(),
        "duplicate_pairs": df.duplicated(subset=["PORTFOLIOCODE", "BENCHMARKCODE"]).sum()
    }

    # Logging
    for key, count in summary.items():
        if count > 0:
            logging.warning(f"[portfolio_benchmark_association] {key.replace('_', ' ').capitalize()}: {count}")

    if not any(summary.values()):
        logging.info("[portfolio_benchmark_association] Validation passed.")

    return pd.DataFrame([summary])


## Populate PBA Table in Snowflake

In [10]:
"""
Creates a staging table, uploads benchmark associations, and merges into the
PortfolioBenchmarkAssociation table in Snowflake.
"""

import os
import pandas as pd
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas

# Connect to Snowflake
conn = connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database="AST_REALESTATE_DB",
    schema="DBO"
)

# Step 1: Set database and schema context, and create staging table
with conn.cursor() as cur:
    """
    Establishes the context to the correct database/schema,
    and creates (or replaces) a temporary staging table to hold
    portfolio-benchmark associations before merging into the target table.
    """
    cur.execute("USE DATABASE AST_REALESTATE_DB")
    cur.execute("USE SCHEMA DBO")

    cur.execute("""
        CREATE OR REPLACE TEMP TABLE STG_PORTFOLIOBENCHMARKASSOCIATION (
            PORTFOLIOCODE VARCHAR,
            BENCHMARKCODE VARCHAR,
            RANK NUMBER,
            RECIPIENTCODE VARCHAR
        )
    """)

# Step 2: Upload assoc_df to staging table
"""
Uploads the DataFrame `assoc_df` to the staging table in Snowflake
using Snowflake's optimized `write_pandas` function.
"""
success, nchunks, nrows, _ = write_pandas(
    conn=conn,
    df=assoc_df,
    table_name="STG_PORTFOLIOBENCHMARKASSOCIATION",
    quote_identifiers=False
)
print(f"Uploaded {nrows} rows to STG_PORTFOLIOBENCHMARKASSOCIATION")

# Step 3: Merge staging data into target table
with conn.cursor() as cur:
    """
    Merges data from the staging table into the target
    `PortfolioBenchmarkAssociation` table.

    - If a (PORTFOLIOCODE, BENCHMARKCODE) pair exists, it updates RANK and RECIPIENTCODE.
    - Otherwise, it inserts a new row.
    """
    cur.execute("""
        MERGE INTO PortfolioBenchmarkAssociation AS target
        USING STG_PORTFOLIOBENCHMARKASSOCIATION AS source
        ON target.PORTFOLIOCODE = source.PORTFOLIOCODE
           AND target.BENCHMARKCODE = source.BENCHMARKCODE

        WHEN MATCHED THEN UPDATE SET
            target.RANK = source.RANK,
            target.RECIPIENTCODE = source.RECIPIENTCODE

        WHEN NOT MATCHED THEN INSERT (
            PORTFOLIOCODE, BENCHMARKCODE, RANK, RECIPIENTCODE
        ) VALUES (
            source.PORTFOLIOCODE, source.BENCHMARKCODE, source.RANK, source.RECIPIENTCODE
        )
    """)
    print("MERGE into PortfolioBenchmarkAssociation completed.")


Uploaded 97 rows to STG_PORTFOLIOBENCHMARKASSOCIATION
MERGE into PortfolioBenchmarkAssociation completed.


# **Data Flow & Architecture Mapping**

## Source to Storage Mapping

| Stage                 | Description                                                                                      |
| --------------------- | ------------------------------------------------------------------------------------------------ |
| **Source**            | `PortfolioGeneralInformation` table in Snowflake (`PORTFOLIOCODE`, `NAME`, etc.)                 |
| **Extraction**        | PGI data is queried using a `SELECT` on Snowflake; loaded into Python as `pgi_df`                |
| **Transformation**    | Benchmark mapping logic parses `NAME` using keyword rules to assign benchmark(s) with ranking    |
|  • `get_benchmarks()` | Uses keyword-based rule matching to select appropriate benchmark(s) for each portfolio           |
|  • `enumerate()`      | Assigns `RANK` (1 = primary, 2 = secondary, etc.) based on order of matching benchmarks          |
| **Validation**        | `validate_portfolio_benchmark_association()` flags missing values, invalid ranks, and duplicates |
| **Storage**           | Enriched `assoc_df` is uploaded via `write_pandas()` into staging, then `MERGE`d into `PBA`      |
| **Consumption**       | Used by benchmark performance lookups, strategy comparables, reporting dashboards, and analytics |


## Data Flow Diagram

          +-----------------------------+
          | PortfolioGeneralInformation|
          |  (Snowflake source)        |
          +-----------------------------+
                      |
            [SELECT * WHERE PORTFOLIOCATEGORY = 'REIT']
                      |
                      v
         +------------------------------------+
         | get_benchmarks(portfolio_name)     |  ← Keyword rule-matching logic
         | → returns benchmark list           |
         +------------------------------------+
                      |
                      v
         +------------------------------------+
         | Build assoc_df                     |  ← Loop: assign RANK and RECIPIENTCODE
         +------------------------------------+
                      |
                      v
         +------------------------------------+
         | validate_portfolio_benchmark_association()  |
         | → Checks nulls, types, duplicates           |
         +------------------------------------+
                      |
                      v
         +------------------------------------+
         | write_pandas() → STG_PBA (temp)    |
         +------------------------------------+
                      |
                      v
         +------------------------------------+
         | MERGE into PortfolioBenchmarkAssociation |
         +------------------------------------+
                      |
                      v
         +------------------------------------+
         | Assette UI / Reporting / Analytics |
         +------------------------------------+


## Transformation Logic Documentation

### Benchmark Mapping

| Trigger Keyword | Benchmarks Assigned            | Notes                                          |
| --------------- | ------------------------------ | ---------------------------------------------- |
| `"Dividend"`    | `REITBENCH003`, `REITBENCH001` | Income-focused portfolios                      |
| `"Utilities"`   | `REITBENCH003`                 | Defensive sector exposure                      |
| `"Tech"`        | `REITBENCH002`                 | Growth-tilted real estate (e.g., data centers) |
| `"Growth"`      | `REITBENCH002`                 | Same as above                                  |
| `"Diversified"` | `REITBENCH001`, `REITBENCH003` | Broad exposure                                 |
| `"Healthcare"`  | `REITBENCH003`                 | Traditional REIT-focused exposure              |
| `"Insurance"`   | `REITBENCH003`                 | Similar sector alignment                       |
| `"Industrial"`  | `REITBENCH001`                 | Aligned with industrial REIT representation    |
| `"Retail"`      | `REITBENCH001`                 | Aligned with traditional retail REITs          |


- Benchmark assignment is non-exclusive — portfolios may receive multiple benchmarks

- RANK is determined by the order of keyword rule matches

### Validation

| Check                  | Description                                                            |
| ---------------------- | ---------------------------------------------------------------------- |
| Required columns       | Must include `PORTFOLIOCODE`, `BENCHMARKCODE`, `RANK`, `RECIPIENTCODE` |
| Missing values         | Flags nulls in key fields                                              |
| Rank sanity            | Ensures `RANK` is ≥1 and a valid integer                               |
| Duplicate combinations | Flags duplicate `(PORTFOLIOCODE, BENCHMARKCODE)` entries               |
| Logging                | Uses Python `logging.warning()` for diagnostics                        |


## Future Enhancements

| Feature                | Description                                                           |
| ---------------------- | --------------------------------------------------------------------- |
| Dynamic Rule Config    | Extract benchmark rules into external YAML or DB table for governance |
| Strategy Tag Join      | Use PGI tags or strategy labels for more precise matching             |
| Manual Overrides       | Allow human-curated benchmark matches via override lookup table       |
| Backfill Audit History | Track `INGESTED_TIMESTAMP`, `ASSIGNED_BY_RULE`, and rule version used |
| Scheduled Refresh      | Add Snowflake Task or cron-based Python job for daily sync            |
| UI Benchmark Preview   | Display benchmark assignments in admin dashboard pre-upload           |
